In [4]:
#Import the necessary packages and libraries
import pandas as pd

In [5]:
#Import the feedback dataset
feedback = pd.read_csv('C:/Users/aidan/Documents/Python projects/Duffl/Feedback Trainer/FeedbackTraining - Feedback.csv')
feedback.head()

,racer,packer,feedback,fault
0,Logan Wiedemann,Keenen Brackett,I was given the wrong items for this order:((((((,P
1,Drew Dieffenbach,Drew Dieffenbach,🔥 racer,R
2,Jaxson Moore,Jaxson Moore,Took extremely long and then dropped it off at...,R
3,Connor Olsen,Isaiah Bowes,My racer got here early and gave me a high fiv...,R
4,Isaiah Bowes,Isaiah Bowes,everything great!,N


In [6]:
for i in range (len(feedback['fault'])):
    if feedback['fault'][i] == 'N' or feedback['fault'][i] == 'G':
        feedback = feedback.drop(index = i)

feedback = feedback.reset_index(drop = True)
feedback.head()

,racer,packer,feedback,fault
0,Logan Wiedemann,Keenen Brackett,I was given the wrong items for this order:((((((,P
1,Drew Dieffenbach,Drew Dieffenbach,🔥 racer,R
2,Jaxson Moore,Jaxson Moore,Took extremely long and then dropped it off at...,R
3,Connor Olsen,Isaiah Bowes,My racer got here early and gave me a high fiv...,R
4,Connor Olsen,Connor Olsen,not big on complaining but he dropped it off l...,R


In [7]:
#Packer/Racer Feedback Dictionaries
PFD = {}
RFD = {}

In [8]:
for i in range (len(feedback['fault'])):
    currFeedback = feedback['feedback'][i].upper()
    # initializing punctuations string
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
 
    # Removing punctuations in string
    # Using loop + punctuation string
    for c in currFeedback:
        if c in punc:
            currFeedback = currFeedback.replace(c, "")
            
    currFeedback = currFeedback.split()
    if feedback['fault'][i] == 'P':
        for j in range (len(currFeedback)):
            if currFeedback[j] in PFD:
                PFD[currFeedback[j]] += 1
            else:
                tempAdd = {currFeedback[j]:1}
                PFD.update(tempAdd)
    elif feedback['fault'][i] == 'R':
        for j in range (len(currFeedback)):
            if currFeedback[j] in RFD:
                RFD[currFeedback[j]] += 1
            else:
                tempAdd = {currFeedback[j]:1}
                RFD.update(tempAdd)
    else:
        continue

In [9]:
Rdf = pd.DataFrame()
Rdf['Words'] = RFD.keys()
Rdf['Frequency'] = RFD.values()
Rdf = Rdf.sort_values('Frequency', ascending = False)
Rdf = Rdf.reset_index(drop = True)
Rdf.head()

,Words,Frequency
0,THE,132
1,MY,128
2,TO,108
3,I,104
4,IT,98


In [10]:
Pdf = pd.DataFrame()
Pdf['Words'] = PFD.keys()
Pdf['Frequency'] = PFD.values()
Pdf = Pdf.sort_values('Frequency', ascending = False)
Pdf = Pdf.reset_index(drop = True)
Pdf.head()

,Words,Frequency
0,I,147
1,THE,128
2,MY,70
3,GOT,57
4,OF,54


In [11]:
crossExamineIntensity = 400
i = 0
while i < crossExamineIntensity:
    currWord = Rdf['Words'][i]
    if currWord in Pdf['Words'].head(crossExamineIntensity).values:
        #print('Dropped: ' + currWord)
        for j in range(len(Pdf['Words'])):
            if Pdf['Words'][j] == currWord:
                if Rdf['Frequency'][i] > Pdf['Frequency'][int(j)]:
                    Pdf = Pdf.drop(index = j)
                else:
                    Rdf = Rdf.drop(index = i)
            else:
                continue
        Rdf = Rdf.reset_index(drop = True)
        Pdf = Pdf.reset_index(drop = True)
        i = 0
    else:
        i += 1
Rdf = Rdf.reset_index(drop = True)
Pdf = Pdf.reset_index(drop = True)

In [12]:
Rdf.head()

,Words,Frequency
0,THE,132
1,MY,128
2,TO,108
3,IT,98
4,ORDER,93


In [13]:
Pdf.head()

,Words,Frequency
0,I,147
1,GOT,57
2,OF,54
3,WRONG,48
4,INSTEAD,37


In [14]:
def predictFeedbackType(df):
    overlapIntensity = 400
    predictionList = []
    for i in range (len(df['feedback'])):
        currFeedback = df['feedback'][i].upper()
        # initializing punctuations string
        punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
     
        # Removing punctuations in string
        # Using loop + punctuation string
        for c in currFeedback:
            if c in punc:
                currFeedback = currFeedback.replace(c, "")

        currFeedback = currFeedback.split()
        PCount = 0
        RCount = 0

        #For each word in the current feedback case, check if it is in a common word in packing or racing feedback
        #if it is then weight it in the count used for predictions
        for j in currFeedback:
            if j in Pdf['Words'].head(overlapIntensity).values:
                PCount += 1 + (int(Pdf["Frequency"].loc[Pdf['Words'] == j]))/100
            elif j in Rdf['Words'].head(overlapIntensity).values:
                RCount += 1 + (int(Rdf["Frequency"].loc[Rdf['Words'] == j]))/100           
        
        if PCount > RCount:
            predictionList.append('P')
            #print("Predicted P")
        elif RCount > PCount:
            predictionList.append('R')
            #print("Predited R")
        elif RCount == PCount:
            predictionList.append('I')
            #print("Inconclusive")
            
    df['Predictions'] = predictionList
    return(df)